disclaimer: this program is created using google colaboratory

# IMPORT DATASET

In [8]:
# @title <p>Essential Import
import os, shutil, json
from PIL import Image
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, random as rd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore")

In [10]:
# @title <p> Mount Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# @title <p> Import Data
DATA_DIR = 'data'
if not os.path.isdir(DATA_DIR) :
  !unzip data.zip &> /dev/null

In [47]:
data_dict = {}
data_list = []

#read each line of file and convert + split data str to int
def convert_data_str_int(data):
    data = data.split(',')
    # data = data[:5] + data[8:-1] + [data[-1].strip()]
    data = data[:5]
    data = [float(x) for x in data]
    return data

#read each file by alphabet name (1 only for each alphabet temporary)
for letter in os.listdir(DATA_DIR):
  data_path = os.path.join(DATA_DIR, letter, f'{letter}-1.txt')
  with open(data_path, 'r') as f:
    data_dict[letter] = f.read().splitlines()

#apply convert function for each alphabet file data
for letter in data_dict.keys():
  data_dict[letter] = [convert_data_str_int(x) for x in data_dict[letter]]

#add labels according to file name
for key, values in data_dict.items():
  for data_row in values:
    data_list.append(data_row + [f'{key}'])

In [48]:
columns = [f'flex{n}' for n in range(1, 6)] + ['label']
data_df = pd.DataFrame(data_list, columns = columns)
data_df[:60]

,finger1,finger2,finger3,finger4,finger5,label
0,0.21,0.27,0.95,0.75,0.72,s
1,0.32,0.10,1.00,0.79,0.81,s
2,0.45,0.02,1.00,0.79,0.83,s
3,0.47,0.02,1.00,0.79,0.84,s
4,0.48,0.02,0.98,0.78,0.83,s
5,0.47,0.03,0.97,0.78,0.82,s
6,0.47,0.03,0.96,0.78,0.81,s
7,0.48,0.03,0.96,0.77,0.81,s
8,0.49,0.02,0.96,0.77,0.82,s
9,0.49,-0.10,0.94,0.77,0.82,s


# TRAINING DATA

In [75]:
# @title <p> One Hot Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data_df['label'])

# Convert integer labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

In [76]:
# @title <p> Split Dataset
TEST_SIZE = 20 #@param{type: 'integer'}
TEST_SIZE = TEST_SIZE/100

X = data_df.drop(columns=['label'], axis=-1)
y = data_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, shuffle=False)

In [93]:
# @title <p> Create Model Architecture
class LSTM_SIBI():
  def __init__(self, X_train, y_train, X_test, y_test, epochs: int=10, batch_size: int=32):
    self.model = Sequential()
    self.train_x_data = np.expand_dims(X_train.values, axis=1)  # Reshape to (samples, 1, features)
    self.train_y_data = y_train
    self.test_x_data = np.expand_dims(X_test.values, axis=1) # Reshape to (samples, 1, features)
    self.test_y_data = y_test
    self.epochs = epochs
    self.batch_size = batch_size

  def model_arch(self):
    self.model.add(LSTM(35, input_shape=(1,5)))
    self.model.add(Dense(10, activation='relu'))
    self.model.add(Dense(26, activation='softmax'))
    return self.model

  def model_train(self):
    self.model_arch()
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    self.model.fit(self.train_x_data, self.train_y_data, epochs = self.epochs, batch_size = self.batch_size)
    return self.model

  def model_run(self):
    self.model_train()

In [94]:
# @title <p> Train Model
EPOCH = 100 #@param{type: 'integer'}
BATCH_SIZE = 32 #@param{type: 'integer'}

ml = LSTM_SIBI(X_train, y_train, X_test, y_test, epochs=EPOCH, batch_size=BATCH_SIZE)
ml.model_run()

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0610 - loss: 3.2570
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0533 - loss: 3.2383
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0467 - loss: 3.2199
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0499 - loss: 3.1899
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0437 - loss: 3.1578
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0418 - loss: 3.1158
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0560 - loss: 3.0545
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0560 - loss: 3.0220
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0555 - loss: 2.9352
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0622 - loss: 2.8834
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1879 - loss: 2.8023
Epoch 12/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

[0.7174493074417114, 1.0]